In [4]:
#hide
import core_constants as cc
import functions as fx
import requests
from bs4 import BeautifulSoup
import lxml
import time
import json
import pandas as pd
import sqlite3 as sql
import recordlinkage
import csv

In [5]:
def handle_allAmerican(years, headers, sleepyTime=5):
    all_players = []
    for y in years: 
        url = 'https://en.wikipedia.org/wiki/{}_College_Football_All-America_Team'.format(y)
        r = requests.get(url, headers=headers)
        aaSoup = BeautifulSoup(r.text, 'lxml')
        for x in aaSoup.find_all("li", class_=""):
            try:
                print(x)
                player = []
                if ((",") in x.text and ("(") in x.text and (")") in x.text and ("{") not in x.text \
                    and ("Archived") not in x.text and ("Gridiron") not in x.text and ("edited") not in x.text \
                    and ("all-purpose") not in x.text and ("Football") not in x.text and ("Foundation") not in x.text):
                    playerInfo = x.text.split(",", 1)
                    #year
                    player.append(int(y))
                    #Name
                    player.append(playerInfo[0])
                    playerLocationAwards = playerInfo[1].split("(",1)

                    #School
                    if ("CONSENSUS" in x.text):
                        playerLocationAwards[0] = playerLocationAwards[0].replace("-- CONSENSUS --", "")
                    elif ("UNANIMOUS" in x.text):
                        playerLocationAwards[0] = playerLocationAwards[0].replace("-- UNANIMOUS --", "")
                    playerLocationAwards[0] = playerLocationAwards[0].lower()
                    if ('miami' in playerLocationAwards[0]):
                        playerLocationAwards[0] = 'miamifl'
                    player.append(playerLocationAwards[0].strip().replace(',',''))
                    #Awards String
                    awardString = playerLocationAwards[1]
                    # This sets AllAmericanBest = 1
                    player.append(1)
                    # No one cares about which All American team they made - and new categories are randomly added
                    # Commenting out the below in case you ever want to bring it back.
                    #coaches (AFCA)
                    
            except:
                print('Error: ' + x)
    time.sleep(sleepyTime)

    final_aaSelections = []
    aa_keys = ['Year', 'PlayerName', 'College', 'AllAmericanBest']

    for list in all_players:
        newdict = {aa_keys[i]: list[i] for i in range(len(aa_keys))}
        newdict['College'] = normalizeAACollege(newdict['College'], cc.get_schoolsList())

        final_aaSelections.append(newdict)
        newdict = {}
    
    
    return final_aaSelections

In [6]:
aa = handle_allAmerican(['2021'], cc.get_header())

In [7]:
print(aa)

[]
